# 4.- Test Environment Train Local

## Sanity-check fine-tuning – YOLO 11 segmentation  

In this section we run a **very short local fine-tuning (1 epoch, small batch)** to confirm that the
Docker environment, dataset paths and CPU/GPU bindings all work **before pushing the full job to Azure**.

> **Prerequisites**  
> | OS | Requirement | Notes |  
> |----|-------------|-------|  
> | **Windows 10/11** | Docker Desktop running | Enable “Use WSL 2 based engine” for best performance. |  
> | **Linux** (Ubuntu, Fedora, …) | Docker Engine ≥ 20.10 | Add your user to the `docker` group or prefix commands with `sudo`. |  
> | **macOS** | Docker Desktop **or** Colima | On Apple Silicon, be sure the image is built for `arm64`. |  

All Ultralytics and PyTorch dependencies live **inside** the Docker image, keeping your host Python environment untouched.

Workflow:

1. **Mount volumes**  
   * `/data/raw` – the dataset folder that contains `images/` and `data.yaml`.  
   * `/train`  – output directory for checkpoints and run artefacts.  
2. **Run `yolo segment train …`** with a minimal configuration:  
   * `epochs=1`, `batch=2` — quick smoke test only.  
   * `device=cpu` — safe default; switch to `device=0` to use your first GPU.  
3. **Stream logs** in real time so you can verify that the training loop runs without errors.


In [2]:
import os
import docker

### 1) Initialise the Docker client

In [47]:
# Create a Docker client
client = docker.from_env()

### 2) Image & volumes


In [48]:
# Docker image name
docker_image = 'yolo11-docker:latest'

In [ ]:
data_local_train_path = os.path.abspath('../data/raw')
local_train_path = os.path.abspath('../train')

# Map host paths → container paths

volumes = {
    data_local_train_path : {'bind' : '/data/raw', 'mode' : 'rw'},
    local_train_path  : {'bind' : '/train', 'mode' : 'rw'},
}

### 3) YOLO command (note the CLI uses key=value syntax)


In [ ]:
command = ['yolo', 'segment', 'train',
           'data= "/data/raw/data.yaml"',
           'model= "/train/pretrained_models/yolo11x-seg.pt"',
           'epochs=1',
           'batch=2',
           'project="train"',
           'name="output"',
           'exist_ok=True',
           'device="cpu"'
           ]

### 4) Run the container and stream logs

In [ ]:
# Run the container and capture the output
try:
    container = client.containers.run(
        docker_image,         # Docker image name
        command,              # Command to execute in container
        volumes=volumes,      # Volumes to mount
        detach=True,          # Run in background (detach mode)
        stdout=True,          # Enable stdout capture
        stderr=True,          # Enable stderr capture
        tty=False             # Disable TTY, since we don’t need interactivity
    )

    print("Container is running...")

    # Collect and print logs in real-time
    for log in container.logs(stream=True):
        print(log.decode('utf-8'), end='')

    # Wait for the container to finish
    container.wait()

    print("Command executed successfully")

except docker.errors.DockerException as e:
    print(f"Error executing Docker command: {e}")

### After the run

* Checkpoints (`*.pt`) and a `results.csv` with per-epoch metrics are saved under  
  `../train/train/output/` by default.  
* Once the smoke test passes, increase `epochs`, `batch`, or add hyper-parameters like `lr0`, `imgsz`, etc.  
* Push the working setup to **Azure ML** (or any cloud) by re-using the same Docker
  image and mounting your blob storage as `/data/raw`.
